<a href="https://colab.research.google.com/github/hmh10098/d2l/blob/main/5_1_Layer_Block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l

     |████████████████████████████████| 83 kB 1.6 MB/s 


In [3]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F

In [10]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.1145, -0.2921,  0.1336, -0.0079, -0.0622, -0.1059,  0.0544, -0.3007,
         -0.0446, -0.2513],
        [-0.0697, -0.4352,  0.1655,  0.1279, -0.0555, -0.2979,  0.1309, -0.3155,
         -0.0948, -0.2672]], grad_fn=<AddmmBackward>)

In [13]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.out = nn.Linear(256, 10)
  
  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))

In [14]:
net = MLP()
net(X)

tensor([[ 0.0012,  0.1720, -0.1660, -0.0834,  0.0187,  0.0222,  0.2639,  0.1959,
          0.0875, -0.0549],
        [-0.0234,  0.0419, -0.1934, -0.0827,  0.0107,  0.0762,  0.2310,  0.2135,
          0.0470, -0.1468]], grad_fn=<AddmmBackward>)

In [19]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [18]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[-0.0299, -0.2931, -0.0759,  0.3000, -0.0273,  0.2107,  0.0033,  0.0202,
          0.2572, -0.2502],
        [-0.0782, -0.0555, -0.0424,  0.1782, -0.1283,  0.1018, -0.1647,  0.0403,
          0.2554, -0.3433]], grad_fn=<AddmmBackward>)

In [21]:
class FixedHiddenMLP(nn.Module):
  def __init__(self, **kwargs):
    super().__init__()
    self.rand_weight = torch.rand((20, 20), requires_grad=False)
    self.linear = nn.Linear(20, 20)
  def forward(self, X):
    X = self.linear(X)
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    X = self.linear(X)
    while X.abs().sum() > 1:
      X /= 2
    return X.sum()

In [24]:
net = FixedHiddenMLP()
net(X)

tensor(0.1315, grad_fn=<SumBackward0>)

In [25]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2126, grad_fn=<SumBackward0>)